In [1]:
#had a lot of issues trying to install the "imputeLCMD" package, so this was how I did it
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("impute", "pcaMethods"))

# install.packages("imputeLCMD")

In [2]:
setwd("/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Input")
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Output')
cur_date = "012722"

library(imputeLCMD)
library(readxl)
library(tidyverse)

#reading in files
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_011822.xlsx", sheet = 2)) 

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:Matrix’:

    which


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
   

In [3]:
# Need to impute missing data, but the distribution needs to be normalized first
ozone_data = ozone_data %>%
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value))) %>%
    #removing all the changes and percent data, since it was throwing infinite (NaN) values and they're 
    #based on the raw data anyways
    filter(grepl("_Change", Variable) == FALSE & grepl("_Percent", Variable) == FALSE) %>%
    #ended up pseudo log transforming the data to prevent infinite (NaN) values
    mutate(Value_log = log(Value)) 

head(ozone_data)

Warning message in ifelse(Value == "NA", NA, as.numeric(Value)):
“NAs introduced by coercion”


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value,Value_log
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.39,1.479329
2,4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,FEV1,4.60,1.526056
3,4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,FEV1,4.39,1.479329
4,4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,FEV1,NA,NA
5,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96,1.085189
6,7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,FEV1,3.08,1.124930


In [4]:
#LAUREN'S VERSION
#Convert cytokine to binary (if the value is missing 0, if the value is present 1)
missing_df <- ozone_data %>% 
    select(-Value) %>%
    mutate(Value_log = ifelse(is.na(Value_log) == TRUE, 0, 1))

head(missing_df)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value_log
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,1
2,4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,FEV1,1
3,4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,FEV1,1
4,4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,FEV1,0
5,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,1
6,7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,FEV1,1


In [5]:
#calculating number of values for each variable, grouped by sample and time_point
totals = missing_df %>%
    group_by(Time_Point, Sample_Type, Variable) %>%
    summarize(Value_Sum = sum(Value_log)) %>%
    #finally calculating the 50% filter
    filter(Value_Sum >= length(unique(ozone_data$Subject_ID)) * 0.5)
head(totals)

`summarise()` has grouped output by 'Time_Point', 'Sample_Type'. You can override using the `.groups` argument.



Time_Point,Sample_Type,Variable,Value_Sum
<chr>,<chr>,<chr>,<dbl>
Post,P,24OH_Chol,24
Post,P,25OH_Chol,24
Post,P,25OH_VitD,24
Post,P,27OH_Chol,24
Post,P,7DHC,24
Post,P,7DHD,24


In [6]:
#creating a new df w/ values present in at least 50% of subjects
new_df = data.frame()
for (i in 1:length(totals$Value_Sum)){
    filtered_df = ozone_data %>%
        filter(Time_Point == totals$Time_Point[i] & totals$Sample_Type[i] == Sample_Type & 
               totals$Variable[i] == Variable)
    new_df = rbind(new_df, filtered_df)
}

head(new_df)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value,Value_log
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Sterol,24OH_Chol,55.512,4.016599
2,7,2,AS_2,AS,P,Post,R,NR,Sterol,24OH_Chol,33.849,3.521909
3,11,3,AS_3,AS,P,Post,NR,R,Sterol,24OH_Chol,29.891,3.397557
4,13,4,AS_4,AS,P,Post,R,NR,Sterol,24OH_Chol,44.330,3.791662
5,15,5,AS_5,AS,P,Post,R,NR,Sterol,24OH_Chol,40.111,3.691651
6,16,6,AS_6,AS,P,Post,NR,NR,Sterol,24OH_Chol,79.289,4.373099


This removed all NA values, so imputation wasn't necessary.

In [7]:
#splitting into separate dfs since the dfs will be converted into a wide format
preimputed_df = new_df %>%
    group_by(Sample_Type, Time_Point) %>%
    group_split()

post_plasma_df = preimputed_df[[1]]
pre_plasma_df = preimputed_df[[2]]
post_sputum_df = preimputed_df[[3]]
pre_sputum_df = preimputed_df[[4]]

In [8]:
wider_post_plasma_df = post_plasma_df %>%
    select(-Value, -Time_Point, -Sample_Type, -Category) %>%
    pivot_wider(names_from = Variable, values_from = Value_log) %>%
    column_to_rownames(var = "Subject_ID") 
head(wider_post_plasma_df)

,Original_Subject_No,Subject_No,Disease_Status,Inflammatory_Responder,Lung_Responder,24OH_Chol,25OH_Chol,25OH_VitD,27OH_Chol,7DHC,⋯,TCC,TCC_Weight,TNFa,TNFb,VEGF,Viability,Zym,Zyme,bEP_Chol,dHLan
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AS_1,4,1,AS,NR,NR,4.016599,2.624886,5.425452,5.854937,-0.09321238,⋯,12.88664,6.181010,0.2240172,-1.308374,2.275213,0.9162907,-2.780621,0.67396436,3.567305,-3.381395
AS_2,7,2,AS,R,NR,3.521909,2.200774,4.660889,5.032873,-0.80968100,⋯,14.78249,5.796301,-0.3150657,-2.002275,2.511259,4.5261270,-2.995732,1.11054086,3.161924,-2.900422
AS_3,11,3,AS,NR,R,3.397557,2.308567,5.364559,5.439331,-1.29462717,⋯,14.73579,6.799647,-0.4787741,-2.604172,1.920995,3.8046605,-3.352407,0.01882175,2.080441,-3.270169
AS_4,13,4,AS,R,NR,3.791662,2.739613,4.473682,5.323537,-1.21402314,⋯,15.04515,6.876099,-0.1156844,-2.156346,2.499970,4.0324692,-4.342806,0.57773633,2.730659,-3.381395
AS_5,15,5,AS,R,NR,3.691651,3.202381,5.019000,5.682021,-0.79407310,⋯,16.00493,7.754306,-0.9899529,-2.283776,2.861146,4.4531838,-3.352407,0.83768033,2.990921,-3.473768
AS_6,16,6,AS,NR,NR,4.373099,2.815768,5.303046,5.369549,-0.64817381,⋯,12.91164,4.813890,0.0047056,-3.569498,2.945379,3.7658405,-2.847312,1.19361939,2.977823,-3.324236


In [65]:
impute.QRILC(wider_post_plasma_df[,6:71])

ERROR: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...): NA/NaN/Inf in 'y'


--------------------------------------------------------